In [ ]:
#!pip install pycryptodome
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Hash import SHA256
import os
import random
import hashlib

In [ ]:
def generate_key(length=32):
  """
  Generates a cryptographically secure key using a combination of:
    - Secure random number generation
    - SHA-256 hashing for improved randomness and strength

  Args:
    length: Desired length of the key in bytes.

  Returns:
    A byte string representing the generated key.
  """

  # Generate a random seed
  seed = random.getrandbits(256).to_bytes(32, byteorder='big')

  # Hash the seed with SHA-256
  hash_obj = hashlib.sha256(seed)
  hashed_seed = hash_obj.digest()

  # Truncate or pad the hashed seed to the desired key length
  if len(hashed_seed) > length:
    key = hashed_seed[:length]
  else:
    key = hashed_seed + os.urandom(length - len(hashed_seed))

  return key

In [ ]:
def pad(data):
    """Pads the data to be a multiple of 16 bytes for AES encryption."""
    return data + (16 - len(data) % 16) * b'\x00'

def unpad(data):
    """Removes the padding from the encrypted data."""
    return data.rstrip(b'\x00')

In [ ]:
def encrypt_aes(key, data):
    """Encrypts the data using AES-128 in CBC mode."""
    cipher = AES.new(key, AES.MODE_CBC, iv=os.urandom(16))
    ciphertext = cipher.encrypt(pad(data))
    return ciphertext, cipher.iv

def decrypt_aes(key, iv, ciphertext):
    """Decrypts the data using AES-128 in CBC mode."""
    cipher = AES.new(key, AES.MODE_CBC, iv=iv)
    plaintext = unpad(cipher.decrypt(ciphertext))
    return plaintext

In [ ]:
def lsb_embed(cover_image_path, secret_message):
    """Embeds the encrypted message into the least significant bits of the cover image."""
    img = Image.open(cover_image_path).convert('RGB')
    pixels = img.load()
    width, height = img.size

    # Generate a key using the secure method
    key = generate_key()

    # Encrypt the secret message
    encrypted_message, iv = encrypt_aes(key, secret_message.encode())

    # Calculate SHA-256 hash of the encrypted message
    hash_obj = SHA256.new()
    hash_obj.update(encrypted_message)
    message_hash = hash_obj.digest()

     # Embed the key, hash, message length, encrypted message, and IV
    message_len = len(encrypted_message)  # Calculate the length of the encrypted message
    data_to_embed = key + message_hash + message_len.to_bytes(4, byteorder='big') + encrypted_message + iv  # Embed message length
    index = 0
    for byte in data_to_embed:
        for j in range(8):
            bit = (byte >> (7 - j)) & 1
            r, g, b = pixels[index % width, index // width]
            r = (r & ~1) | bit
            pixels[index % width, index // width] = (r, g, b)
            index += 1

    img.save("/content/drive/MyDrive/Output_SHA256/Image.png")

In [ ]:
def lsb_extract(stego_image_path):
    """Extracts the encrypted message from the stego image."""
    img = Image.open(stego_image_path)
    pixels = img.load()
    width, height = img.size

    # Extract the key, hash, message length, message, and IV
    key_len = 32  # Assuming key length is 32 bytes
    hash_len = 32  # SHA-256 hash is 32 bytes
    iv_len = 16  # IV length is 16 bytes

    extracted_data = bytearray()
    index = 0

    # Calculate total bits to extract for key, hash, message length, message and IV
    # + message_len to ensure we extract enough bits for the entire message
    # Ensure we extract enough bits for the entire message
    total_bits_to_extract = 8 * (key_len + hash_len + 4)  # Key, Hash, and Message Length
    for _ in range(total_bits_to_extract):
        if index >= width * height:
            return None  # or raise an exception
        r, _, _ = pixels[index % width, index // width]
        extracted_data.append(r & 1)
        index += 1

    # Convert the initial bitstream to bytes (for key, hash, message length)
    extracted_data_bytes = bytes(int("".join(map(str, extracted_data[i:i + 8])), 2) for i in range(0, len(extracted_data), 8))

    # Extract key, hash, message length from the bytes
    extracted_key = extracted_data_bytes[:key_len]
    extracted_hash = extracted_data_bytes[key_len:key_len + hash_len]
    message_len = int.from_bytes(extracted_data_bytes[key_len + hash_len:key_len + hash_len + 4], byteorder='big')

    # Now calculate total bits for message and IV
    remaining_bits_to_extract = 8 * (message_len + iv_len)
    extracted_message_data = bytearray()
    for _ in range(remaining_bits_to_extract):
        if index >= width * height:
            return None  # or raise an exception
        r, _, _ = pixels[index % width, index // width]
        extracted_message_data.append(r & 1)
        index += 1

    # Convert message and IV bitstream to bytes
    extracted_message_bytes = bytes(int("".join(map(str, extracted_message_data[i:i + 8])), 2) for i in range(0, len(extracted_message_data), 8))


    # Extract message and IV
    extracted_message = extracted_message_bytes[:message_len]
    extracted_iv = extracted_message_bytes[message_len:]

    # Decrypt the message
    decrypted_message = decrypt_aes(extracted_key, extracted_iv, extracted_message)

    # Calculate SHA-256 hash of the extracted message
    hash_obj = SHA256.new()
    hash_obj.update(extracted_message)
    calculated_hash = hash_obj.digest()

    if calculated_hash == extracted_hash:
        return decrypted_message.decode()
    else:
        return None

In [ ]:
# Example usage
secret_message = "This is a secret message."

lsb_embed("/content/drive/MyDrive/Paper2Images_Input/Image.png", secret_message)
extracted_message = lsb_extract("/content/drive/MyDrive/Output_SHA256/Image.png")

if extracted_message:
    print("Extracted message:", extracted_message)
else:
    print("Message extraction failed or message has been tampered with.")

Extracted message: This is a secret message.


In [ ]:
def lsb_embed(cover_image_path, secret_message):
    """Embeds the encrypted message into the least significant bits of the cover image."""
    img = Image.open(cover_image_path).convert('RGB')
    pixels = img.load()
    width, height = img.size

    # Generate a key using the secure method
    key = generate_key()

    # Encrypt the secret message
    encrypted_message, iv = encrypt_aes(key, secret_message.encode())

    # Calculate SHA-256 hash of the encrypted message
    hash_obj = SHA256.new()
    hash_obj.update(encrypted_message)
    message_hash = hash_obj.digest()

     # Embed the key, hash, message length, encrypted message, and IV
    message_len = len(encrypted_message)  # Calculate the length of the encrypted message
    data_to_embed = key + message_hash + message_len.to_bytes(4, byteorder='big') + encrypted_message + iv  # Embed message length
    index = 0
    for byte in data_to_embed:
        for j in range(8):
            bit = (byte >> (7 - j)) & 1
            r, g, b = pixels[index % width, index // width]
            r = (r & ~1) | bit
            pixels[index % width, index // width] = (r, g, b)
            index += 1

    img.save("/content/drive/MyDrive/Output_SHA256/Image1.png")
# Example usage
secret_message = "This$3 3D World is a secret message."

lsb_embed("/content/drive/MyDrive/Paper2Images_Input/Image1.png", secret_message)
extracted_message = lsb_extract("/content/drive/MyDrive/Output_SHA256/Image1.png")

if extracted_message:
    print("Extracted message:", extracted_message)
else:
    print("Message extraction failed or message has been tampered with.")

Extracted message: This$3 3D World is a secret message.


In [ ]:
def lsb_embed(cover_image_path, secret_message):
    """Embeds the encrypted message into the least significant bits of the cover image."""
    img = Image.open(cover_image_path).convert('RGB')
    pixels = img.load()
    width, height = img.size

    # Generate a key using the secure method
    key = generate_key()

    # Encrypt the secret message
    encrypted_message, iv = encrypt_aes(key, secret_message.encode())

    # Calculate SHA-256 hash of the encrypted message
    hash_obj = SHA256.new()
    hash_obj.update(encrypted_message)
    message_hash = hash_obj.digest()

     # Embed the key, hash, message length, encrypted message, and IV
    message_len = len(encrypted_message)  # Calculate the length of the encrypted message
    data_to_embed = key + message_hash + message_len.to_bytes(4, byteorder='big') + encrypted_message + iv  # Embed message length
    index = 0
    for byte in data_to_embed:
        for j in range(8):
            bit = (byte >> (7 - j)) & 1
            r, g, b = pixels[index % width, index // width]
            r = (r & ~1) | bit
            pixels[index % width, index // width] = (r, g, b)
            index += 1

    img.save("/content/drive/MyDrive/Output_SHA256/Image2.png")
# Example usage
secret_message = """for _ in range(bits_to_extract):
            if index >= width * height:
                break  # Stop if we reach the end of the image
            r, _, _ = pixels[index % width, index // width]
            extracted_data.append(r & 1)
            index += 1"""

lsb_embed("/content/drive/MyDrive/Paper2Images_Input/Image2.png", secret_message)
extracted_message = lsb_extract("/content/drive/MyDrive/Output_SHA256/Image2.png")

if extracted_message:
    print("Extracted message:", extracted_message)
else:
    print("Message extraction failed or message has been tampered with.")

Extracted message: for _ in range(bits_to_extract):
            if index >= width * height:
                break  # Stop if we reach the end of the image
            r, _, _ = pixels[index % width, index // width]
            extracted_data.append(r & 1)
            index += 1


In [ ]:
def lsb_embed(cover_image_path, secret_message):
    """Embeds the encrypted message into the least significant bits of the cover image."""
    img = Image.open(cover_image_path).convert('RGB')
    pixels = img.load()
    width, height = img.size

    # Generate a key using the secure method
    key = generate_key()

    # Encrypt the secret message
    encrypted_message, iv = encrypt_aes(key, secret_message.encode())

    # Calculate SHA-256 hash of the encrypted message
    hash_obj = SHA256.new()
    hash_obj.update(encrypted_message)
    message_hash = hash_obj.digest()

     # Embed the key, hash, message length, encrypted message, and IV
    message_len = len(encrypted_message)  # Calculate the length of the encrypted message
    data_to_embed = key + message_hash + message_len.to_bytes(4, byteorder='big') + encrypted_message + iv  # Embed message length
    index = 0
    for byte in data_to_embed:
        for j in range(8):
            bit = (byte >> (7 - j)) & 1
            r, g, b = pixels[index % width, index // width]
            r = (r & ~1) | bit
            pixels[index % width, index // width] = (r, g, b)
            index += 1

    img.save("/content/drive/MyDrive/Output_SHA256/Image3.png")
# Example usage
secret_message = """# Convert the complete bitstream to bytes after processing all images
    extracted_data_bytes = bytes(int("".join(map(str, extracted_data[i:i + 8])), 2) for i in range(0, len(extracted_data), 8))

    # Ensure we have enough data to extract the message and IV
    if len(extracted_data_bytes) < message_len + iv_len:
        return None  # or raise an exception

    # Extract message and IV from the final bytes
    extracted_message = extracted_data_bytes[:message_len]
    extracted_iv = extracted_data_bytes[message_len:message_len + iv_len]
    """

lsb_embed("/content/drive/MyDrive/Paper2Images_Input/Image3.png", secret_message)
extracted_message = lsb_extract("/content/drive/MyDrive/Output_SHA256/Image3.png")


if extracted_message:
    print("Extracted message:", extracted_message)
else:
    print("Message extraction failed or message has been tampered with.")

Extracted message: # Convert the complete bitstream to bytes after processing all images
    extracted_data_bytes = bytes(int("".join(map(str, extracted_data[i:i + 8])), 2) for i in range(0, len(extracted_data), 8))

    # Ensure we have enough data to extract the message and IV
    if len(extracted_data_bytes) < message_len + iv_len:
        return None  # or raise an exception

    # Extract message and IV from the final bytes
    extracted_message = extracted_data_bytes[:message_len]
    extracted_iv = extracted_data_bytes[message_len:message_len + iv_len]
    


In [ ]:
def lsb_embed(cover_image_path, secret_message):
    """Embeds the encrypted message into the least significant bits of the cover image."""
    img = Image.open(cover_image_path).convert('RGB')
    pixels = img.load()
    width, height = img.size

    # Generate a key using the secure method
    key = generate_key()

    # Encrypt the secret message
    encrypted_message, iv = encrypt_aes(key, secret_message.encode())

    # Calculate SHA-256 hash of the encrypted message
    hash_obj = SHA256.new()
    hash_obj.update(encrypted_message)
    message_hash = hash_obj.digest()

     # Embed the key, hash, message length, encrypted message, and IV
    message_len = len(encrypted_message)  # Calculate the length of the encrypted message
    data_to_embed = key + message_hash + message_len.to_bytes(4, byteorder='big') + encrypted_message + iv  # Embed message length
    index = 0
    for byte in data_to_embed:
        for j in range(8):
            bit = (byte >> (7 - j)) & 1
            r, g, b = pixels[index % width, index // width]
            r = (r & ~1) | bit
            pixels[index % width, index // width] = (r, g, b)
            index += 1

    img.save("/content/drive/MyDrive/Output_SHA256/Image4.png")
# Example usage
secret_message = """Extracted_data_bytes = bytes(int("".join(map(str, extracted_data[i:i + 8])), 2) for i in range(0, len(extracted_data), 8))

    # Ensure we have enough data to extract the message and IV
    if len(extracted_data_bytes) < message_len + iv_len:
        return None  # or raise an exception

    # Extract message and IV from the final bytes
    extracted_message = extracted_data_bytes[:message_len]
    extracted_iv = extracted_data_bytes[message_len:message_len + iv_len]
    """

lsb_embed("/content/drive/MyDrive/Paper2Images_Input/Image4.png", secret_message)
extracted_message = lsb_extract("/content/drive/MyDrive/Output_SHA256/Image4.png")


if extracted_message:
    print("Extracted message:", extracted_message)
else:
    print("Message extraction failed or message has been tampered with.")

Extracted message: Extracted_data_bytes = bytes(int("".join(map(str, extracted_data[i:i + 8])), 2) for i in range(0, len(extracted_data), 8))

    # Ensure we have enough data to extract the message and IV
    if len(extracted_data_bytes) < message_len + iv_len:
        return None  # or raise an exception

    # Extract message and IV from the final bytes
    extracted_message = extracted_data_bytes[:message_len]
    extracted_iv = extracted_data_bytes[message_len:message_len + iv_len]
    


In [ ]:
def lsb_embed(cover_image_path, secret_message):
    """Embeds the encrypted message into the least significant bits of the cover image."""
    img = Image.open(cover_image_path).convert('RGB')
    pixels = img.load()
    width, height = img.size

    # Generate a key using the secure method
    key = generate_key()

    # Encrypt the secret message
    encrypted_message, iv = encrypt_aes(key, secret_message.encode())

    # Calculate SHA-256 hash of the encrypted message
    hash_obj = SHA256.new()
    hash_obj.update(encrypted_message)
    message_hash = hash_obj.digest()

     # Embed the key, hash, message length, encrypted message, and IV
    message_len = len(encrypted_message)  # Calculate the length of the encrypted message
    data_to_embed = key + message_hash + message_len.to_bytes(4, byteorder='big') + encrypted_message + iv  # Embed message length
    index = 0
    for byte in data_to_embed:
        for j in range(8):
            bit = (byte >> (7 - j)) & 1
            r, g, b = pixels[index % width, index // width]
            r = (r & ~1) | bit
            pixels[index % width, index // width] = (r, g, b)
            index += 1

    img.save("/content/drive/MyDrive/Output_SHA256/Image5.png")
# Example usage
secret_message = """After processing all images
    extracted_data_bytes = bytes(int("".join(map(str, extracted_data[i:i + 8])), 2) for i in range(0, len(extracted_data), 8))

    # Ensure we have enough data to extract the message and IV
    if len(extracted_data_bytes) < message_len + iv_len:
        return None  # or raise an exception

    # Extract message and IV from the final bytes
    extracted_message = extracted_data_bytes[:message_len]
    extracted_iv = extracted_data_bytes[message_len:message_len + iv_len]
    """

lsb_embed("/content/drive/MyDrive/Paper2Images_Input/Image5.png", secret_message)
extracted_message = lsb_extract("/content/drive/MyDrive/Output_SHA256/Image5.png")


if extracted_message:
    print("Extracted message:", extracted_message)
else:
    print("Message extraction failed or message has been tampered with.")

Extracted message: After processing all images
    extracted_data_bytes = bytes(int("".join(map(str, extracted_data[i:i + 8])), 2) for i in range(0, len(extracted_data), 8))

    # Ensure we have enough data to extract the message and IV
    if len(extracted_data_bytes) < message_len + iv_len:
        return None  # or raise an exception

    # Extract message and IV from the final bytes
    extracted_message = extracted_data_bytes[:message_len]
    extracted_iv = extracted_data_bytes[message_len:message_len + iv_len]
    


In [ ]:
def lsb_embed(cover_image_path, secret_message):
    """Embeds the encrypted message into the least significant bits of the cover image."""
    img = Image.open(cover_image_path).convert('RGB')
    pixels = img.load()
    width, height = img.size

    # Generate a key using the secure method
    key = generate_key()

    # Encrypt the secret message
    encrypted_message, iv = encrypt_aes(key, secret_message.encode())

    # Calculate SHA-256 hash of the encrypted message
    hash_obj = SHA256.new()
    hash_obj.update(encrypted_message)
    message_hash = hash_obj.digest()

     # Embed the key, hash, message length, encrypted message, and IV
    message_len = len(encrypted_message)  # Calculate the length of the encrypted message
    data_to_embed = key + message_hash + message_len.to_bytes(4, byteorder='big') + encrypted_message + iv  # Embed message length
    index = 0
    for byte in data_to_embed:
        for j in range(8):
            bit = (byte >> (7 - j)) & 1
            r, g, b = pixels[index % width, index // width]
            r = (r & ~1) | bit
            pixels[index % width, index // width] = (r, g, b)
            index += 1

    img.save("/content/drive/MyDrive/Output_SHA256/Image6.png")
# Example usage
secret_message = """Ex-$xtracted_data_bytes = bytes(int("".join(map(str, extracted_data[i:i + 8])), 2) for i in range(0, len(extracted_data), 8))

    # Ensure we have enough data to extract the message and IV
    if len(extracted_data_bytes) < message_len + iv_len:
        return None  # or raise an exception

    # Extract message and IV from the final bytes
    extracted_message = extracted_data_bytes[:message_len]
    extracted_iv = extracted_data_bytes[message_len:message_len + iv_len]
    """

lsb_embed("/content/drive/MyDrive/Paper2Images_Input/Image6.png", secret_message)
extracted_message = lsb_extract("/content/drive/MyDrive/Output_SHA256/Image6.png")


if extracted_message:
    print("Extracted message:", extracted_message)
else:
    print("Message extraction failed or message has been tampered with.")

Extracted message: Ex-$xtracted_data_bytes = bytes(int("".join(map(str, extracted_data[i:i + 8])), 2) for i in range(0, len(extracted_data), 8))

    # Ensure we have enough data to extract the message and IV
    if len(extracted_data_bytes) < message_len + iv_len:
        return None  # or raise an exception

    # Extract message and IV from the final bytes
    extracted_message = extracted_data_bytes[:message_len]
    extracted_iv = extracted_data_bytes[message_len:message_len + iv_len]
    


In [ ]:
def lsb_embed(cover_image_path, secret_message):
    """Embeds the encrypted message into the least significant bits of the cover image."""
    img = Image.open(cover_image_path).convert('RGB')
    pixels = img.load()
    width, height = img.size

    # Generate a key using the secure method
    key = generate_key()

    # Encrypt the secret message
    encrypted_message, iv = encrypt_aes(key, secret_message.encode())

    # Calculate SHA-256 hash of the encrypted message
    hash_obj = SHA256.new()
    hash_obj.update(encrypted_message)
    message_hash = hash_obj.digest()

     # Embed the key, hash, message length, encrypted message, and IV
    message_len = len(encrypted_message)  # Calculate the length of the encrypted message
    data_to_embed = key + message_hash + message_len.to_bytes(4, byteorder='big') + encrypted_message + iv  # Embed message length
    index = 0
    for byte in data_to_embed:
        for j in range(8):
            bit = (byte >> (7 - j)) & 1
            r, g, b = pixels[index % width, index // width]
            r = (r & ~1) | bit
            pixels[index % width, index // width] = (r, g, b)
            index += 1

    img.save("/content/drive/MyDrive/Output_SHA256/Image7.png")
# Example usage
secret_message = """# Ensure we have enough data to extract the message and IV
    if len(extracted_data_bytes) < message_len + iv_len:
        return None  # or raise an exception

    # Extract message and IV from the final bytes
    extracted_message = extracted_data_bytes[:message_len]
    extracted_iv = extracted_data_bytes[message_len:message_len + iv_len]
    """

lsb_embed("/content/drive/MyDrive/Paper2Images_Input/Image7.png", secret_message)
extracted_message = lsb_extract("/content/drive/MyDrive/Output_SHA256/Image7.png")


if extracted_message:
    print("Extracted message:", extracted_message)
else:
    print("Message extraction failed or message has been tampered with.")

Extracted message: # Ensure we have enough data to extract the message and IV
    if len(extracted_data_bytes) < message_len + iv_len:
        return None  # or raise an exception

    # Extract message and IV from the final bytes
    extracted_message = extracted_data_bytes[:message_len]
    extracted_iv = extracted_data_bytes[message_len:message_len + iv_len]
    


In [ ]:
def lsb_embed(cover_image_path, secret_message):
    """Embeds the encrypted message into the least significant bits of the cover image."""
    img = Image.open(cover_image_path).convert('RGB')
    pixels = img.load()
    width, height = img.size

    # Generate a key using the secure method
    key = generate_key()

    # Encrypt the secret message
    encrypted_message, iv = encrypt_aes(key, secret_message.encode())

    # Calculate SHA-256 hash of the encrypted message
    hash_obj = SHA256.new()
    hash_obj.update(encrypted_message)
    message_hash = hash_obj.digest()

     # Embed the key, hash, message length, encrypted message, and IV
    message_len = len(encrypted_message)  # Calculate the length of the encrypted message
    data_to_embed = key + message_hash + message_len.to_bytes(4, byteorder='big') + encrypted_message + iv  # Embed message length
    index = 0
    for byte in data_to_embed:
        for j in range(8):
            bit = (byte >> (7 - j)) & 1
            r, g, b = pixels[index % width, index // width]
            r = (r & ~1) | bit
            pixels[index % width, index // width] = (r, g, b)
            index += 1

    img.save("/content/drive/MyDrive/Output_SHA256/Image8.png")
# Example usage
secret_message = """Make sure its secret if you know we have enough data to extract the message and IV
    if len(extracted_data_bytes) < message_len + iv_len:
        return None  # or raise an exception."""

lsb_embed("/content/drive/MyDrive/Paper2Images_Input/Image8.png", secret_message)
extracted_message = lsb_extract("/content/drive/MyDrive/Output_SHA256/Image8.png")


if extracted_message:
    print("Extracted message:", extracted_message)
else:
    print("Message extraction failed or message has been tampered with.")

Extracted message: Make sure its secret if you know we have enough data to extract the message and IV
    if len(extracted_data_bytes) < message_len + iv_len:
        return None  # or raise an exception.


In [ ]:
def lsb_embed(cover_image_path, secret_message):
    """Embeds the encrypted message into the least significant bits of the cover image."""
    img = Image.open(cover_image_path).convert('RGB')
    pixels = img.load()
    width, height = img.size

    # Generate a key using the secure method
    key = generate_key()

    # Encrypt the secret message
    encrypted_message, iv = encrypt_aes(key, secret_message.encode())

    # Calculate SHA-256 hash of the encrypted message
    hash_obj = SHA256.new()
    hash_obj.update(encrypted_message)
    message_hash = hash_obj.digest()

     # Embed the key, hash, message length, encrypted message, and IV
    message_len = len(encrypted_message)  # Calculate the length of the encrypted message
    data_to_embed = key + message_hash + message_len.to_bytes(4, byteorder='big') + encrypted_message + iv  # Embed message length
    index = 0
    for byte in data_to_embed:
        for j in range(8):
            bit = (byte >> (7 - j)) & 1
            r, g, b = pixels[index % width, index // width]
            r = (r & ~1) | bit
            pixels[index % width, index // width] = (r, g, b)
            index += 1

    img.save("/content/drive/MyDrive/Output_SHA256/Image9.png")
# Example usage
secret_message = """Extract message and IV from the final bytes
    extracted_message = extracted_data_bytes[:message_len]
    extracted_iv = extracted_data_bytes[message_len:message_len + iv_len]
    """

lsb_embed("/content/drive/MyDrive/Paper2Images_Input/Image9.png", secret_message)
extracted_message = lsb_extract("/content/drive/MyDrive/Output_SHA256/Image9.png")


if extracted_message:
    print("Extracted message:", extracted_message)
else:
    print("Message extraction failed or message has been tampered with.")

Extracted message: Extract message and IV from the final bytes
    extracted_message = extracted_data_bytes[:message_len]
    extracted_iv = extracted_data_bytes[message_len:message_len + iv_len]
    


In [ ]:
def lsb_embed(cover_image_path, secret_message):
    """Embeds the encrypted message into the least significant bits of the cover image."""
    img = Image.open(cover_image_path).convert('RGB')
    pixels = img.load()
    width, height = img.size

    # Generate a key using the secure method
    key = generate_key()

    # Encrypt the secret message
    encrypted_message, iv = encrypt_aes(key, secret_message.encode())

    # Calculate SHA-256 hash of the encrypted message
    hash_obj = SHA256.new()
    hash_obj.update(encrypted_message)
    message_hash = hash_obj.digest()

     # Embed the key, hash, message length, encrypted message, and IV
    message_len = len(encrypted_message)  # Calculate the length of the encrypted message
    data_to_embed = key + message_hash + message_len.to_bytes(4, byteorder='big') + encrypted_message + iv  # Embed message length
    index = 0
    for byte in data_to_embed:
        for j in range(8):
            bit = (byte >> (7 - j)) & 1
            r, g, b = pixels[index % width, index // width]
            r = (r & ~1) | bit
            pixels[index % width, index // width] = (r, g, b)
            index += 1

    img.save("/content/drive/MyDrive/Output_SHA256/Image10.png")
# Example usage
secret_message = """Extracted_message = extracted_data_bytes[:message_len]
    extracted_iv = extracted_data_bytes[message_len:message_len + iv_len]
    """

lsb_embed("/content/drive/MyDrive/Paper2Images_Input/Image10.png", secret_message)
extracted_message = lsb_extract("/content/drive/MyDrive/Output_SHA256/Image10.png")


if extracted_message:
    print("Extracted message:", extracted_message)
else:
    print("Message extraction failed or message has been tampered with.")

Extracted message: Extracted_message = extracted_data_bytes[:message_len]
    extracted_iv = extracted_data_bytes[message_len:message_len + iv_len]
    
